# Chapter 6 - Point Feature Detection

In the last blog article, we've seen how filters can be used to find edges in an image. However, we can also use filters to detect features in an image. 

## Template Matching
To illustrate this concept, let's have a look at an image with some simple structures on it. We can copy a small region of our image and use it as a filter / template. When we slide this filter over the image, we'll get a so called *correlation map* that indicates how well the template matched the original picture at each possible position. As we might expect, the point of best correlation is found at the brightest pixel at exactly the position where we extracted the template originally. 

![Template matching](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/template_matching.png)
*Figure 1: Template, Detected template and correlation map. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

To find the difference between the template and the image, we can use **cross correlation**, hence the name **cross correlation map**. In normalized cross correlation (NCC), we interpret pixels as vectors and take their cross product divided by the vectors lengths. If the images / vectors are equal, their NCC will be 1. If they are perpendicular, the NCC is 0 - a NCC of -1 suggest that the vectors are exact opposites of each other. 
While NCC is very robust, the problem with NCC is the computational complexity: It is computationally expensive to calculate NCC, hence it is seldomly used in practice. 

![Normalized Cross Correlation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/cross_correlation.png)
*Figure 2: Cross correlation formula. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

An alternative and much faster similarity measurement is the **Sum of Squared Differences** (SSD). SSD, as the name sais, takes the piecewise difference of each pixel value, squares it and sums the results. 

![Sum of Squared Differences](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/sum_of_squared_differences.png)
*Figure 3: Sum of Squared Differences. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

Even cheaper than SSD is the third similarity measurement, called **Sum of Absolute Differences** (SAD). It works like SSD but replaces the square with the absolute function, making it even faster to compute but also less robust. 

![Sum of Absolute Differences](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/sum_of_absolute_differences.png)
*Figure 4: Sum of Absolute Differences. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

All of these difference measurements are prone to intensity variations. To account for this, we can substract the average intensity of the two images (typically caused by additive illumination changes) from each image. We then get what's called **Zero-mean Normalized Cross-Corrleation (ZNCC)**, **Zero-mean Sum of Squared Differences ZSSD)** and **Zero-mean Sum of Absolute Differences ZSAD)**. 

![Zero-mean similarities](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/zero_mean_similarities.png)
*Figure 5: Zero-mean similarities. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

The last error measurement we will have a look in this article is called **Census Transform**. CT maps each pixel in the template patch to either 1 (white) or 0 (black) depending whether it is brighter or darker than the central pixel. Then, it flattens the pixels clockwise into a vector called *bit string*. For a template with dimensions w * w, the resulting bit string vector would be of length w^2 - 1 (minus the central pixel). We can then compare two patches by converting them to a bit string and apply *Hamming distance*, which is the total number of bits that are different. This can be done by performing an XOR operation on both bit strings. 
This process has the advantage that it is very fast since neither roots nor divisions are required. Since the pixels are compared to the central pixel, the Census Transform is automatically also invariant to monotonic intensity changes. 

![Census Transform](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/census_transform.png)
*Figure 6: Census Transform. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

While template matching is cool, it's applications are limited to cases where we match on a pixel level. We can't match objects that are just similar to the pixel. We even fail if the overall brightness of the image changed or the rotation / orientation slightly changed. As we compare exact pixel values, we also need to have the exact pixel equivalent to the filter inside the image. 

## Point-feature extraction
For visual odometry, we want to find and extract features from an image that we can use as templates in another image. This way, we can match features between two images and calculate the relative motion between them. 

Matching keypoints can also be used for other applications like *Panorama Stitching*, *Object recognition*, *3D reconstruction* or *place recognition*. The problem is always the same: Given one image, we want to match another image of the same scene but taken under different environmental conditions. 

### Panorama stitching
As an easy example, let's consider panorama stitching where we have two images taken side by side with a certain overlap, and we want to combine these two images to get a panorama stitch. We can dothis in three steps:
- 1.) Find features in both images
- 2.) For each feature, find the corresponding pair in the other picture
- 3.) Overlap images such that the pairs are aligned as good as possible

![Panorama Stitching](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/panorama_stitching.png)
*Figure 7: Panorama Stitching. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

The hard part is to detect the same points independently in both images. We need to have repeateable feature detectors, meaning that we have to re-detect the same features from one images in another. 
Secondly, we have to find the corresponding features in both images distinctively. To do so, we use descriptors. Descriptors are a description of a pixel and the features around it that uniquely identify the pixel without ambiguity. The descriptor needs to be robust against geometric and illumination changes such that we can find the same descriptor in a different image. Such geometrical changes can be: Translation, Rotation, Scaling and perspective changes. 

An illumination change is a simple, affine transformation of a pixels value by a constant factor *B*. It can easily be overcome by dividing an image by its average intensity value. 
Rotation invariation is harder to achieve. We could - for example - always rotate our detected features such that the most dominant lines are horizontal, but we will see more efficient methods later in this article. 

When we detect features, we usually have two different methods: **corner detection** and **blob detection**. 
A corner is a region of contrast that changes significantly in two perpendicular directions. It has a very high localization accuracy but are often hard to distinct. In contrast, a Blob is an image region that differs significantly from its neighbours - like a small but dominant pattern. Blogs are less localization accurate than corners but are more distinct and therefore easier to redetect. 

## Moravec Corner Detector
Moravec proposed a corner detectoor in which we slide a window into any direction and measure the intensity change. A corner is found when movement into at least two directions results in a large change in intensity. 

![Moravec Corner Detector](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/moravec_corner_detector.png)
*Figure 7: Moravec Corner Detector. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

We consider a window / patch at position (x,y) and move it to a new location (x+dx, y+dy). we then copute the sum of squared differences between the two patches is large for two different dx and dy pairs, we consider the region (x,y) to contain a corner. 

![SSD of Moravec Corner Detector](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/ssd_moravec_corner_detector.png)
*Figure 8: SSD of Moravec Corner Detector. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

While Moravecs approach is intuitive to understand, it is also computationally expensive, since for each pixel location, multiple different patch positions have to be calculated and compared to each other. 

## Harris Corner Detector
Harris implements Moravecs corner detection without the need to physicalls shift the patch window around. It does so by looking at the patch and calculating the gradients, so the derivatives. 
Harris approximates the shift (dx, dy) using the first taylor expansion: I(x + dx, y + dy) = I(x, y) + Ix(x, y)dx + Iy(x, y)dy. 
The SSD of (dx, dy) can therefore be approximated by taking the sum of squared differences between the images derivative in x-direction (Ix) and the images derivative in y-direction (Iy).

![SSD of Harris Corner Detector](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/ssd_harris_corner_detector.png)
*Figure 9: SSD of Harris Corner Detector. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*

To implement this formula, we can write the SSD in matrix form. Note that in the following formula, M is a so called *second moment matrix* containing pixel-wise products of their respective image patch. 

![SSD of Harris in Matrix representation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_6/harris_corner_detector_matrix.png)
*Figure 9: SSD of Harris in Matrix representation. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/05_feature_detection_1.pdf)*